# QAOA Algorithm – 8-qubit optimization with backend timing comparison

In [ ]:
import time

from qiskit import QuantumCircuit

from ariadne import simulate


# Create 8-qubit QAOA circuit for MaxCut
def create_qaoa_circuit(n_qubits=8, p=1):
    qc = QuantumCircuit(n_qubits, n_qubits)

    # Initial superposition
    for i in range(n_qubits):
        qc.h(i)

    # QAOA layers
    for _layer in range(p):
        # Problem Hamiltonian (MaxCut on ring graph)
        for i in range(n_qubits):
            qc.cx(i, (i + 1) % n_qubits)
            qc.rz(0.5, (i + 1) % n_qubits)
            qc.cx(i, (i + 1) % n_qubits)

        # Mixer Hamiltonian
        for i in range(n_qubits):
            qc.rx(0.3, i)

    qc.measure_all()
    return qc


# Create the circuit
qaoa_circuit = create_qaoa_circuit(8, p=2)
print(f"QAOA circuit: {qaoa_circuit.num_qubits} qubits, {qaoa_circuit.depth()} depth")

In [ ]:
# Test across different backends and measure timing
backends = ["stim", "qiskit", "mps", "tensor_network"]
results = {}

for backend in backends:
    try:
        start_time = time.time()
        result = simulate(qaoa_circuit, shots=1000, backend=backend)
        end_time = time.time()

        results[backend] = {
            "time": end_time - start_time,
            "counts": result.counts,
            "backend_used": result.backend_used.value,
        }

        print(f"{backend}: {end_time - start_time:.4f}s, backend: {result.backend_used.value}")
    except Exception as e:
        print(f"{backend}: Failed - {e}")
        results[backend] = {"error": str(e)}

In [ ]:
# Display timing table
print("\nBackend Timing Comparison:")
print("=" * 40)
for backend, data in results.items():
    if "error" not in data:
        print(f"{backend:15} | {data['time']:8.4f}s | {data['backend_used']}")
    else:
        print(f"{backend:15} | {'FAILED':>8} | {data['error']}")

Notice how different backends handle the same QAOA circuit with varying performance. This demonstrates the importance of backend selection for different algorithm types.